# 멀티모달 태아 성별 예측 모델 비교 실험

본 노트북은 태아 초음파 이미지와 임상 텍스트 정보를 활용한 멀티모달 성별 예측 모델의 성능 비교 실험을 수행합니다.

## 실험 목표
논문계획서에 따라 다음 모델들의 성능을 비교 분석합니다:

### 비교 모델군
1. **단일 모달 기반 모델**
   - 텍스트 전용 모델 (임상 수치 데이터)
   - 이미지 전용 모델 (CNN 기반)
   - ViT 전용 모델 (Vision Transformer)

2. **멀티모달 모델**
   - ViT + 텍스트 결합 모델 (Early Fusion)
   - ViT + 텍스트 결합 모델 (Late Fusion) 
   - ViT + 텍스트 결합 모델 (Attention Fusion)

## 실험 설정
- **Task**: 이진 분류 (남성/여성)
- **데이터**: 합성 초음파 이미지 + 임상 텍스트 데이터
- **평가 지표**: Accuracy, Precision, Recall, F1-Score, ROC-AUC
- **모델 저장**: 각 모델을 파일로 저장하여 추후 로드 가능

In [ ]:
# 필수 라이브러리 Import
import os
import json
import pickle
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# 딥러닝 프레임워크
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report

# Transformer 및 ViT
from transformers import ViTImageProcessor, ViTForImageClassification, ViTModel
import transformers

# 이미지 처리
from PIL import Image
import cv2

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 모델 저장/로드
import joblib

# 기본 설정
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("🚀 라이브러리 Import 완료!")
print(f"💻 사용 디바이스: {device}")
print(f"🔥 PyTorch 버전: {torch.__version__}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"🔢 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")

In [ ]:
# 합성 태아 초음파 데이터 생성기
class FetalDataGenerator:
    """태아 성별 예측을 위한 합성 데이터 생성기"""
    
    def __init__(self, num_samples=1000, image_size=(224, 224), random_seed=42):
        self.num_samples = num_samples
        self.image_size = image_size
        self.random_seed = random_seed
        np.random.seed(random_seed)
        
    def generate_clinical_data(self):
        """임상 텍스트 데이터 생성 (수치형 특징)"""
        
        # 성별 라벨 생성 (0: 여성, 1: 남성)
        gender_labels = np.random.randint(0, 2, self.num_samples)
        
        clinical_features = []
        
        for i in range(self.num_samples):
            gender = gender_labels[i]
            
            # 성별에 따른 특징 분포 설정
            if gender == 1:  # 남성
                gestational_age = np.random.normal(28, 4)  # 주수
                head_circumference = np.random.normal(265, 15)  # 머리둘레(mm)
                femur_length = np.random.normal(52, 8)  # 대퇴골 길이(mm)
                estimated_weight = np.random.normal(1800, 300)  # 예상 체중(g)
                heart_rate = np.random.normal(145, 10)  # 심박수
                biparietal_diameter = np.random.normal(72, 5)  # 양두정간경(mm)
            else:  # 여성
                gestational_age = np.random.normal(28, 4)
                head_circumference = np.random.normal(260, 15)
                femur_length = np.random.normal(50, 8)
                estimated_weight = np.random.normal(1750, 300)
                heart_rate = np.random.normal(150, 10)
                biparietal_diameter = np.random.normal(70, 5)
            
            # 값 범위 제한
            gestational_age = np.clip(gestational_age, 20, 40)
            head_circumference = np.clip(head_circumference, 200, 350)
            femur_length = np.clip(femur_length, 30, 80)
            estimated_weight = np.clip(estimated_weight, 800, 3500)
            heart_rate = np.clip(heart_rate, 120, 180)
            biparietal_diameter = np.clip(biparietal_diameter, 50, 100)
            
            clinical_features.append([
                gestational_age,
                head_circumference,
                femur_length,
                estimated_weight,
                heart_rate,
                biparietal_diameter
            ])
        
        feature_names = [
            'gestational_age',  # 주수
            'head_circumference',  # 머리둘레
            'femur_length',  # 대퇴골 길이
            'estimated_weight',  # 예상 체중
            'heart_rate',  # 심박수
            'biparietal_diameter'  # 양두정간경
        ]
        
        return np.array(clinical_features), gender_labels, feature_names
    
    def generate_ultrasound_images(self, gender_labels):
        """초음파 이미지 데이터 생성 (합성)"""
        
        images = []
        
        for i in range(self.num_samples):
            gender = gender_labels[i]
            
            # 기본 배경 생성
            image = np.random.rand(self.image_size[0], self.image_size[1], 3) * 50
            
            # 태아 영역 시뮬레이션
            center_x, center_y = self.image_size[0]//2, self.image_size[1]//2
            
            # 성별에 따른 특징적 패턴 추가
            if gender == 1:  # 남성
                # 남성적 특징 패턴 (더 선명한 구조물)
                cv2.circle(image, (center_x, center_y), 40, (120, 120, 120), -1)
                cv2.rectangle(image, (center_x-15, center_y+20), (center_x+15, center_y+50), (80, 80, 80), -1)
            else:  # 여성
                # 여성적 특징 패턴 (더 부드러운 구조물)
                cv2.circle(image, (center_x, center_y), 35, (100, 100, 100), -1)
                cv2.ellipse(image, (center_x, center_y+30), (25, 15), 0, 0, 360, (90, 90, 90), -1)
            
            # 노이즈 추가 (초음파 특성)
            noise = np.random.normal(0, 20, image.shape)
            image = np.clip(image + noise, 0, 255).astype(np.uint8)
            
            # 초음파 특성 필터 적용
            image = cv2.GaussianBlur(image, (5, 5), 1)
            
            images.append(image)
        
        return np.array(images)
    
    def generate_dataset(self):
        """전체 데이터셋 생성"""
        
        print(f"📊 {self.num_samples}개 샘플 생성 중...")
        
        # 임상 데이터 생성
        clinical_data, gender_labels, feature_names = self.generate_clinical_data()
        
        # 초음파 이미지 생성  
        ultrasound_images = self.generate_ultrasound_images(gender_labels)
        
        dataset = {
            'clinical_data': clinical_data,
            'ultrasound_images': ultrasound_images,
            'gender_labels': gender_labels,
            'feature_names': feature_names,
            'class_names': ['Female', 'Male']
        }
        
        print(f"✅ 데이터 생성 완료!")
        print(f"   - 임상 데이터 형태: {clinical_data.shape}")
        print(f"   - 이미지 데이터 형태: {ultrasound_images.shape}")
        print(f"   - 성별 분포: 여성 {(gender_labels==0).sum()}명, 남성 {(gender_labels==1).sum()}명")
        
        return dataset

# 데이터셋 생성
print("🔄 합성 태아 데이터 생성 중...")
data_generator = FetalDataGenerator(num_samples=2000, random_seed=42)
dataset = data_generator.generate_dataset()

In [ ]:
# 데이터 전처리 및 시각화

def preprocess_and_visualize_data(dataset):
    """데이터 전처리 및 시각화"""
    
    # 데이터 추출
    clinical_data = dataset['clinical_data']
    images = dataset['ultrasound_images']
    labels = dataset['gender_labels']
    feature_names = dataset['feature_names']
    class_names = dataset['class_names']
    
    # 1. 임상 데이터 표준화
    scaler = StandardScaler()
    clinical_data_scaled = scaler.fit_transform(clinical_data)
    
    # 2. 데이터 분할 (train/val/test)
    # 먼저 train+val과 test로 분할
    X_clinical_temp, X_clinical_test, X_images_temp, X_images_test, y_temp, y_test = train_test_split(
        clinical_data_scaled, images, labels, test_size=0.2, random_state=42, stratify=labels
    )
    
    # train+val을 train과 val로 분할
    X_clinical_train, X_clinical_val, X_images_train, X_images_val, y_train, y_val = train_test_split(
        X_clinical_temp, X_images_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
    )
    
    # 데이터 분포 시각화
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('📊 데이터 분포 및 특성 분석', fontsize=16, fontweight='bold')
    
    # 1. 성별 분포
    axes[0, 0].bar(class_names, [np.sum(labels == 0), np.sum(labels == 1)], 
                   color=['pink', 'lightblue'], alpha=0.8)
    axes[0, 0].set_title('성별 분포', fontweight='bold')
    axes[0, 0].set_ylabel('샘플 수')
    
    # 2. 임상 특징 분포 (성별별)
    clinical_df = pd.DataFrame(clinical_data, columns=feature_names)
    clinical_df['gender'] = ['Female' if l == 0 else 'Male' for l in labels]
    
    # 주요 특징 선택하여 박스플롯
    key_features = ['gestational_age', 'head_circumference', 'femur_length']
    for i, feature in enumerate(key_features):
        sns.boxplot(data=clinical_df, x='gender', y=feature, ax=axes[0, i+1])
        axes[0, i+1].set_title(f'{feature}', fontweight='bold')
    
    # 3. 샘플 이미지 시각화
    sample_indices = np.random.choice(len(images), 6, replace=False)
    for i, idx in enumerate(sample_indices):
        row, col = i // 3, i % 3
        axes[1, col].imshow(images[idx])
        axes[1, col].set_title(f'{class_names[labels[idx]]} (Sample {idx})', fontweight='bold')
        axes[1, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 상관관계 분석
    plt.figure(figsize=(10, 8))
    correlation_matrix = pd.DataFrame(clinical_data, columns=feature_names).corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, cbar_kws={'label': 'Correlation'})
    plt.title('임상 특징 간 상관관계', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # 전처리된 데이터 반환
    preprocessed_data = {
        'X_clinical_train': X_clinical_train,
        'X_clinical_val': X_clinical_val, 
        'X_clinical_test': X_clinical_test,
        'X_images_train': X_images_train,
        'X_images_val': X_images_val,
        'X_images_test': X_images_test,
        'y_train': y_train,
        'y_val': y_val,
        'y_test': y_test,
        'scaler': scaler,
        'feature_names': feature_names,
        'class_names': class_names
    }
    
    print("✅ 데이터 전처리 완료!")
    print(f"   📈 훈련 세트: {len(y_train)} 샘플")
    print(f"   🔍 검증 세트: {len(y_val)} 샘플") 
    print(f"   🧪 테스트 세트: {len(y_test)} 샘플")
    
    return preprocessed_data

# 데이터 전처리 실행
data = preprocess_and_visualize_data(dataset)

In [ ]:
# 1. 단일 모달 모델 정의

class TextOnlyModel(nn.Module):
    """텍스트(임상 데이터) 전용 분류 모델"""
    
    def __init__(self, input_dim, hidden_dims=[128, 64], num_classes=2, dropout=0.3):
        super(TextOnlyModel, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, num_classes))
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

class CNNImageModel(nn.Module):
    """CNN 기반 이미지 전용 분류 모델"""
    
    def __init__(self, num_classes=2):
        super(CNNImageModel, self).__init__()
        
        self.features = nn.Sequential(
            # Conv Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((7, 7))
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

class ViTOnlyModel(nn.Module):
    """ViT 기반 이미지 전용 분류 모델"""
    
    def __init__(self, model_name="google/vit-base-patch16-224", num_classes=2):
        super(ViTOnlyModel, self).__init__()
        
        # ViT 모델 로드
        self.vit = ViTModel.from_pretrained(model_name)
        
        # 분류 헤드
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.vit.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
        
        # ViT의 일부 레이어 동결 (선택사항)
        for param in list(self.vit.parameters())[:-4]:  # 마지막 4개 레이어만 학습
            param.requires_grad = False
    
    def forward(self, pixel_values):
        outputs = self.vit(pixel_values=pixel_values)
        pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)
        return logits

# 모델 인스턴스 생성
print("🔨 단일 모달 모델들 생성 중...")

# 텍스트 전용 모델
text_model = TextOnlyModel(input_dim=data['X_clinical_train'].shape[1]).to(device)

# CNN 이미지 모델
cnn_model = CNNImageModel().to(device)

# ViT 전용 모델
vit_model = ViTOnlyModel().to(device)

print(f"✅ 단일 모달 모델 생성 완료!")
print(f"   📊 텍스트 모델 파라미터: {sum(p.numel() for p in text_model.parameters()):,}")
print(f"   🖼️  CNN 모델 파라미터: {sum(p.numel() for p in cnn_model.parameters()):,}")
print(f"   🤖 ViT 모델 파라미터: {sum(p.numel() for p in vit_model.parameters()):,}")

In [ ]:
# 2. 멀티모달 모델 정의

class EarlyFusionModel(nn.Module):
    """조기 융합 멀티모달 모델 (특징 단계에서 결합)"""
    
    def __init__(self, clinical_input_dim, vit_model_name="google/vit-base-patch16-224", num_classes=2):
        super(EarlyFusionModel, self).__init__()
        
        # ViT 백본
        self.vit = ViTModel.from_pretrained(vit_model_name)
        
        # 임상 데이터 인코더
        self.clinical_encoder = nn.Sequential(
            nn.Linear(clinical_input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 512),
            nn.ReLU()
        )
        
        # ViT 특징 변환
        self.vit_projection = nn.Sequential(
            nn.Linear(self.vit.config.hidden_size, 512),
            nn.ReLU()
        )
        
        # 융합된 특징 분류기
        self.fusion_classifier = nn.Sequential(
            nn.Linear(512 + 512, 256),  # ViT + 임상 데이터
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, pixel_values, clinical_data):
        # ViT 특징 추출
        vit_outputs = self.vit(pixel_values=pixel_values)
        vit_features = self.vit_projection(vit_outputs.pooler_output)
        
        # 임상 데이터 인코딩
        clinical_features = self.clinical_encoder(clinical_data)
        
        # 특징 융합
        fused_features = torch.cat([vit_features, clinical_features], dim=1)
        
        # 분류
        logits = self.fusion_classifier(fused_features)
        return logits

class LateFusionModel(nn.Module):
    """후기 융합 멀티모달 모델 (예측 단계에서 결합)"""
    
    def __init__(self, clinical_input_dim, vit_model_name="google/vit-base-patch16-224", num_classes=2):
        super(LateFusionModel, self).__init__()
        
        # ViT 분류기
        self.vit = ViTModel.from_pretrained(vit_model_name)
        self.vit_classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.vit.config.hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        
        # 임상 데이터 분류기
        self.clinical_classifier = nn.Sequential(
            nn.Linear(clinical_input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )
        
        # 융합 가중치
        self.fusion_weights = nn.Parameter(torch.tensor([0.5, 0.5]))
        
    def forward(self, pixel_values, clinical_data):
        # ViT 예측
        vit_outputs = self.vit(pixel_values=pixel_values)
        vit_logits = self.vit_classifier(vit_outputs.pooler_output)
        
        # 임상 데이터 예측
        clinical_logits = self.clinical_classifier(clinical_data)
        
        # 가중 융합
        weights = torch.softmax(self.fusion_weights, dim=0)
        fused_logits = weights[0] * vit_logits + weights[1] * clinical_logits
        
        return fused_logits

class AttentionFusionModel(nn.Module):
    """어텐션 기반 멀티모달 모델"""
    
    def __init__(self, clinical_input_dim, vit_model_name="google/vit-base-patch16-224", num_classes=2):
        super(AttentionFusionModel, self).__init__()
        
        # ViT 백본
        self.vit = ViTModel.from_pretrained(vit_model_name)
        
        # 임상 데이터 인코더
        self.clinical_encoder = nn.Sequential(
            nn.Linear(clinical_input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 512)
        )
        
        # ViT 특징 변환
        self.vit_projection = nn.Linear(self.vit.config.hidden_size, 512)
        
        # 크로스 어텐션
        self.cross_attention = nn.MultiheadAttention(embed_dim=512, num_heads=8, batch_first=True)
        
        # 분류기
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, pixel_values, clinical_data):
        # ViT 특징 추출
        vit_outputs = self.vit(pixel_values=pixel_values)
        vit_features = self.vit_projection(vit_outputs.pooler_output)  # [batch_size, 512]
        
        # 임상 데이터 인코딩
        clinical_features = self.clinical_encoder(clinical_data)  # [batch_size, 512]
        
        # 어텐션을 위한 차원 확장
        vit_features = vit_features.unsqueeze(1)  # [batch_size, 1, 512]
        clinical_features = clinical_features.unsqueeze(1)  # [batch_size, 1, 512]
        
        # 크로스 어텐션 적용 (clinical을 query, vit를 key/value로 사용)
        attended_features, attention_weights = self.cross_attention(
            clinical_features, vit_features, vit_features
        )
        
        # 차원 축소
        attended_features = attended_features.squeeze(1)  # [batch_size, 512]
        
        # 분류
        logits = self.classifier(attended_features)
        return logits

# 멀티모달 모델 인스턴스 생성
print("🔨 멀티모달 모델들 생성 중...")

clinical_dim = data['X_clinical_train'].shape[1]

# Early Fusion 모델
early_fusion_model = EarlyFusionModel(clinical_input_dim=clinical_dim).to(device)

# Late Fusion 모델  
late_fusion_model = LateFusionModel(clinical_input_dim=clinical_dim).to(device)

# Attention Fusion 모델
attention_fusion_model = AttentionFusionModel(clinical_input_dim=clinical_dim).to(device)

print(f"✅ 멀티모달 모델 생성 완료!")
print(f"   🔗 Early Fusion 파라미터: {sum(p.numel() for p in early_fusion_model.parameters()):,}")
print(f"   🔗 Late Fusion 파라미터: {sum(p.numel() for p in late_fusion_model.parameters()):,}")
print(f"   🎯 Attention Fusion 파라미터: {sum(p.numel() for p in attention_fusion_model.parameters()):,}")

In [ ]:
# 3. 데이터로더 및 훈련 함수

class MultimodalDataset(Dataset):
    """멀티모달 데이터셋 클래스"""
    
    def __init__(self, clinical_data, images, labels, processor=None):
        self.clinical_data = torch.FloatTensor(clinical_data)
        self.images = images
        self.labels = torch.LongTensor(labels)
        self.processor = processor
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        clinical = self.clinical_data[idx]
        image = self.images[idx]
        label = self.labels[idx]
        
        # 이미지 전처리
        if self.processor:
            # ViT 프로세서 사용
            image = Image.fromarray(image)
            image = self.processor(image, return_tensors="pt")['pixel_values'].squeeze(0)
        else:
            # CNN을 위한 일반 전처리
            image = torch.FloatTensor(image).permute(2, 0, 1) / 255.0
        
        return {
            'clinical': clinical,
            'image': image, 
            'label': label
        }

def create_dataloaders(data, batch_size=32, use_vit_processor=True):
    """데이터로더 생성"""
    
    processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224") if use_vit_processor else None
    
    train_dataset = MultimodalDataset(
        data['X_clinical_train'], data['X_images_train'], data['y_train'], processor
    )
    val_dataset = MultimodalDataset(
        data['X_clinical_val'], data['X_images_val'], data['y_val'], processor
    )
    test_dataset = MultimodalDataset(
        data['X_clinical_test'], data['X_images_test'], data['y_test'], processor
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    return train_loader, val_loader, test_loader

def train_model(model, train_loader, val_loader, num_epochs=10, lr=1e-4, model_name="model"):
    """모델 훈련 함수"""
    
    print(f"🚀 {model_name} 훈련 시작...")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    best_val_acc = 0.0
    
    for epoch in range(num_epochs):
        # 훈련 모드
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch in train_loader:
            clinical = batch['clinical'].to(device)
            image = batch['image'].to(device) 
            labels = batch['label'].to(device)
            
            optimizer.zero_grad()
            
            # 모델 타입에 따른 forward pass
            if isinstance(model, TextOnlyModel):
                outputs = model(clinical)
            elif isinstance(model, (CNNImageModel, ViTOnlyModel)):
                outputs = model(image)
            else:  # 멀티모달 모델
                outputs = model(image, clinical)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # 검증 모드
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch in val_loader:
                clinical = batch['clinical'].to(device)
                image = batch['image'].to(device)
                labels = batch['label'].to(device)
                
                if isinstance(model, TextOnlyModel):
                    outputs = model(clinical)
                elif isinstance(model, (CNNImageModel, ViTOnlyModel)):
                    outputs = model(image)
                else:
                    outputs = model(image, clinical)
                
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # 메트릭 계산
        train_acc = 100.0 * train_correct / train_total
        val_acc = 100.0 * val_correct / val_total
        
        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(val_acc)
        
        # 학습률 업데이트
        scheduler.step()
        
        # 최고 성능 모델 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f'models/{model_name}_best.pth')
        
        # 진행 상황 출력
        if epoch % 2 == 0 or epoch == num_epochs - 1:
            print(f"   Epoch {epoch+1}/{num_epochs}: "
                  f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%, "
                  f"Val Loss: {val_loss/len(val_loader):.4f}")
    
    print(f"✅ {model_name} 훈련 완료! 최고 검증 정확도: {best_val_acc:.2f}%")
    
    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies,
        'best_val_acc': best_val_acc
    }

# 모델 저장 디렉토리 생성
os.makedirs('models', exist_ok=True)

# 데이터로더 생성 (ViT용)
train_loader_vit, val_loader_vit, test_loader_vit = create_dataloaders(data, batch_size=16, use_vit_processor=True)

# 데이터로더 생성 (CNN용) 
train_loader_cnn, val_loader_cnn, test_loader_cnn = create_dataloaders(data, batch_size=32, use_vit_processor=False)

print("✅ 데이터로더 생성 완료!")

In [ ]:
# 4. 모델 훈련 실행

# 훈련 설정
num_epochs = 8
learning_rate = 1e-4

training_results = {}

print("🔥 모든 모델 훈련 시작!")
print("="*60)

# 1. 텍스트 전용 모델 훈련
print("\\n📊 텍스트 전용 모델 훈련")
text_results = train_model(
    text_model, 
    train_loader_vit,  # 임상 데이터는 동일하므로 아무거나 사용 가능
    val_loader_vit, 
    num_epochs=num_epochs,
    lr=learning_rate,
    model_name="text_only"
)
training_results['text_only'] = text_results

# 2. CNN 이미지 모델 훈련  
print("\\n🖼️ CNN 이미지 모델 훈련")
cnn_results = train_model(
    cnn_model,
    train_loader_cnn,
    val_loader_cnn,
    num_epochs=num_epochs,
    lr=learning_rate,
    model_name="cnn_image"
)
training_results['cnn_image'] = cnn_results

# 3. ViT 전용 모델 훈련
print("\\n🤖 ViT 전용 모델 훈련") 
vit_results = train_model(
    vit_model,
    train_loader_vit,
    val_loader_vit,
    num_epochs=num_epochs,
    lr=learning_rate*0.5,  # ViT는 더 낮은 학습률 사용
    model_name="vit_only"
)
training_results['vit_only'] = vit_results

# 4. Early Fusion 멀티모달 모델 훈련
print("\\n🔗 Early Fusion 멀티모달 모델 훈련")
early_fusion_results = train_model(
    early_fusion_model,
    train_loader_vit,
    val_loader_vit,
    num_epochs=num_epochs,
    lr=learning_rate*0.5,
    model_name="early_fusion"
)
training_results['early_fusion'] = early_fusion_results

# 5. Late Fusion 멀티모달 모델 훈련
print("\\n🔗 Late Fusion 멀티모달 모델 훈련")
late_fusion_results = train_model(
    late_fusion_model,
    train_loader_vit,
    val_loader_vit,
    num_epochs=num_epochs,
    lr=learning_rate*0.5,
    model_name="late_fusion"
)
training_results['late_fusion'] = late_fusion_results

# 6. Attention Fusion 멀티모달 모델 훈련
print("\\n🎯 Attention Fusion 멀티모달 모델 훈련")
attention_fusion_results = train_model(
    attention_fusion_model,
    train_loader_vit,
    val_loader_vit,
    num_epochs=num_epochs,
    lr=learning_rate*0.5,
    model_name="attention_fusion"
)
training_results['attention_fusion'] = attention_fusion_results

print("\\n" + "="*60)
print("🎉 모든 모델 훈련 완료!")
print("="*60)

# 훈련 결과 요약
print("\\n📋 훈련 결과 요약:")
print("-"*50)
for model_name, results in training_results.items():
    print(f"{model_name:20}: 최고 검증 정확도 {results['best_val_acc']:.2f}%")

# 훈련 히스토리 저장
with open('models/training_results.json', 'w') as f:
    # numpy array를 list로 변환하여 JSON 직렬화 가능하게 만듦
    json_results = {}
    for model_name, results in training_results.items():
        json_results[model_name] = {
            'train_losses': results['train_losses'],
            'val_losses': results['val_losses'],
            'val_accuracies': results['val_accuracies'],
            'best_val_acc': results['best_val_acc']
        }
    json.dump(json_results, f, indent=2)

print("\\n💾 훈련 결과가 저장되었습니다!")

In [ ]:
# 5. 모델 평가 및 성능 비교

def evaluate_model(model, test_loader, model_name):
    """모델 평가 함수"""
    
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for batch in test_loader:
            clinical = batch['clinical'].to(device)
            image = batch['image'].to(device)
            labels = batch['label'].to(device)
            
            # 모델 타입에 따른 forward pass
            if isinstance(model, TextOnlyModel):
                outputs = model(clinical)
            elif isinstance(model, (CNNImageModel, ViTOnlyModel)):
                outputs = model(image)
            else:  # 멀티모달 모델
                outputs = model(image, clinical)
            
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    # 메트릭 계산
    accuracy = accuracy_score(all_labels, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
    
    # ROC AUC (이진 분류)
    probabilities_positive = [prob[1] for prob in all_probabilities]  # 양성 클래스 확률
    roc_auc = roc_auc_score(all_labels, probabilities_positive)
    
    results = {
        'model_name': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'predictions': all_predictions,
        'labels': all_labels,
        'probabilities': all_probabilities
    }
    
    return results

# 모델 로드 및 평가
models = {
    'text_only': text_model,
    'cnn_image': cnn_model, 
    'vit_only': vit_model,
    'early_fusion': early_fusion_model,
    'late_fusion': late_fusion_model,
    'attention_fusion': attention_fusion_model
}

# 최적 모델 가중치 로드
for model_name, model in models.items():
    try:
        model.load_state_dict(torch.load(f'models/{model_name}_best.pth'))
        print(f"✅ {model_name} 최적 가중치 로드 완료")
    except FileNotFoundError:
        print(f"⚠️ {model_name} 가중치 파일을 찾을 수 없습니다. 현재 가중치를 사용합니다.")

print("\\n🧪 모든 모델 테스트 평가 시작...")

evaluation_results = {}

# 텍스트 모델 평가 (ViT 로더 사용, 임상 데이터만 필요)
evaluation_results['text_only'] = evaluate_model(text_model, test_loader_vit, 'Text Only')

# CNN 모델 평가
evaluation_results['cnn_image'] = evaluate_model(cnn_model, test_loader_cnn, 'CNN Image')

# ViT 모델 평가  
evaluation_results['vit_only'] = evaluate_model(vit_model, test_loader_vit, 'ViT Only')

# 멀티모달 모델들 평가
evaluation_results['early_fusion'] = evaluate_model(early_fusion_model, test_loader_vit, 'Early Fusion')
evaluation_results['late_fusion'] = evaluate_model(late_fusion_model, test_loader_vit, 'Late Fusion') 
evaluation_results['attention_fusion'] = evaluate_model(attention_fusion_model, test_loader_vit, 'Attention Fusion')

print("✅ 모든 모델 평가 완료!")

# 결과 테이블 생성
results_df = pd.DataFrame([
    {
        'Model': result['model_name'],
        'Accuracy': f"{result['accuracy']:.4f}",
        'Precision': f"{result['precision']:.4f}",
        'Recall': f"{result['recall']:.4f}",
        'F1-Score': f"{result['f1_score']:.4f}",
        'ROC-AUC': f"{result['roc_auc']:.4f}"
    }
    for result in evaluation_results.values()
])

print("\\n📊 모델 성능 비교 결과:")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

In [ ]:
# 6. 결과 시각화 및 분석

def create_performance_visualization(training_results, evaluation_results):
    """성능 시각화 함수"""
    
    # 서브플롯 생성
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('🎯 멀티모달 태아 성별 예측 모델 성능 비교', fontsize=18, fontweight='bold')
    
    # 1. 훈련 손실 비교
    axes[0, 0].set_title('📉 Training Loss Curves', fontweight='bold', fontsize=14)
    for model_name, results in training_results.items():
        axes[0, 0].plot(results['train_losses'], label=model_name, linewidth=2, alpha=0.8)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Training Loss')
    axes[0, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 검증 정확도 비교
    axes[0, 1].set_title('📈 Validation Accuracy Curves', fontweight='bold', fontsize=14)
    for model_name, results in training_results.items():
        axes[0, 1].plot(results['val_accuracies'], label=model_name, linewidth=2, alpha=0.8)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Validation Accuracy (%)')
    axes[0, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 테스트 성능 바 차트
    model_names = [result['model_name'] for result in evaluation_results.values()]
    accuracies = [result['accuracy'] for result in evaluation_results.values()]
    f1_scores = [result['f1_score'] for result in evaluation_results.values()]
    
    x = np.arange(len(model_names))
    width = 0.35
    
    axes[0, 2].bar(x - width/2, accuracies, width, label='Accuracy', alpha=0.8)
    axes[0, 2].bar(x + width/2, f1_scores, width, label='F1-Score', alpha=0.8)
    axes[0, 2].set_title('🎯 Test Performance Comparison', fontweight='bold', fontsize=14)
    axes[0, 2].set_ylabel('Score')
    axes[0, 2].set_xticks(x)
    axes[0, 2].set_xticklabels(model_names, rotation=45, ha='right')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # 값 표시
    for i, (acc, f1) in enumerate(zip(accuracies, f1_scores)):
        axes[0, 2].text(i - width/2, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
        axes[0, 2].text(i + width/2, f1 + 0.01, f'{f1:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # 4. ROC AUC 비교
    roc_aucs = [result['roc_auc'] for result in evaluation_results.values()]
    colors = plt.cm.Set3(np.linspace(0, 1, len(model_names)))
    
    bars = axes[1, 0].bar(model_names, roc_aucs, color=colors, alpha=0.8)
    axes[1, 0].set_title('📊 ROC-AUC Comparison', fontweight='bold', fontsize=14)
    axes[1, 0].set_ylabel('ROC-AUC Score')
    axes[1, 0].set_xticklabels(model_names, rotation=45, ha='right')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 값 표시
    for bar, auc in zip(bars, roc_aucs):
        axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                       f'{auc:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 5. 혼동행렬 (최고 성능 모델)
    best_model_name = max(evaluation_results.keys(), key=lambda x: evaluation_results[x]['accuracy'])
    best_result = evaluation_results[best_model_name]
    
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(best_result['labels'], best_result['predictions'])
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 1],
                xticklabels=data['class_names'], yticklabels=data['class_names'])
    axes[1, 1].set_title(f'🎯 Confusion Matrix\\n({best_result["model_name"]})', fontweight='bold', fontsize=14)
    axes[1, 1].set_xlabel('Predicted')
    axes[1, 1].set_ylabel('Actual')
    
    # 6. 모델 복잡도 vs 성능
    model_params = {
        'Text Only': sum(p.numel() for p in text_model.parameters()),
        'CNN Image': sum(p.numel() for p in cnn_model.parameters()),
        'ViT Only': sum(p.numel() for p in vit_model.parameters()),
        'Early Fusion': sum(p.numel() for p in early_fusion_model.parameters()),
        'Late Fusion': sum(p.numel() for p in late_fusion_model.parameters()),
        'Attention Fusion': sum(p.numel() for p in attention_fusion_model.parameters())
    }
    
    params_list = [model_params[name] for name in model_names]
    
    scatter = axes[1, 2].scatter(params_list, accuracies, c=colors, s=100, alpha=0.8)
    axes[1, 2].set_title('🔧 Model Complexity vs Performance', fontweight='bold', fontsize=14)
    axes[1, 2].set_xlabel('Number of Parameters')
    axes[1, 2].set_ylabel('Test Accuracy')
    axes[1, 2].grid(True, alpha=0.3)
    axes[1, 2].set_xscale('log')
    
    # 모델 이름 표시
    for i, (name, params, acc) in enumerate(zip(model_names, params_list, accuracies)):
        axes[1, 2].annotate(name, (params, acc), xytext=(5, 5), textcoords='offset points',
                           fontsize=9, alpha=0.8)
    
    plt.tight_layout()
    plt.savefig('models/performance_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

def create_detailed_analysis(evaluation_results):
    """상세 분석 리포트 생성"""
    
    print("\\n" + "="*80)
    print("📋 상세 분석 리포트")
    print("="*80)
    
    # 1. 단일 모달 vs 멀티모달 성능 비교
    unimodal_models = ['text_only', 'cnn_image', 'vit_only']
    multimodal_models = ['early_fusion', 'late_fusion', 'attention_fusion']
    
    unimodal_avg_acc = np.mean([evaluation_results[model]['accuracy'] for model in unimodal_models])
    multimodal_avg_acc = np.mean([evaluation_results[model]['accuracy'] for model in multimodal_models])
    
    print(f"\\n🔍 단일 모달 vs 멀티모달 성능:")
    print(f"   📊 단일 모달 평균 정확도: {unimodal_avg_acc:.4f}")
    print(f"   🔗 멀티모달 평균 정확도: {multimodal_avg_acc:.4f}")
    print(f"   📈 성능 향상: {(multimodal_avg_acc - unimodal_avg_acc)*100:.2f}%p")
    
    # 2. 최고 성능 모델 분석
    best_model = max(evaluation_results.keys(), key=lambda x: evaluation_results[x]['accuracy'])
    best_acc = evaluation_results[best_model]['accuracy']
    
    print(f"\\n🏆 최고 성능 모델:")
    print(f"   모델: {evaluation_results[best_model]['model_name']}")
    print(f"   정확도: {best_acc:.4f}")
    print(f"   F1-Score: {evaluation_results[best_model]['f1_score']:.4f}")
    print(f"   ROC-AUC: {evaluation_results[best_model]['roc_auc']:.4f}")
    
    # 3. 융합 전략 비교
    print(f"\\n🔗 멀티모달 융합 전략 비교:")
    for model in multimodal_models:
        result = evaluation_results[model]
        print(f"   {result['model_name']:15}: Acc={result['accuracy']:.4f}, F1={result['f1_score']:.4f}")
    
    # 4. 모달리티별 기여도 분석
    text_acc = evaluation_results['text_only']['accuracy']
    vit_acc = evaluation_results['vit_only']['accuracy']
    best_multimodal_acc = max([evaluation_results[model]['accuracy'] for model in multimodal_models])
    
    print(f"\\n📊 모달리티별 기여도:")
    print(f"   텍스트 단독: {text_acc:.4f}")
    print(f"   ViT 단독: {vit_acc:.4f}")  
    print(f"   최고 멀티모달: {best_multimodal_acc:.4f}")
    print(f"   텍스트 대비 향상: {(best_multimodal_acc - text_acc)*100:.2f}%p")
    print(f"   ViT 대비 향상: {(best_multimodal_acc - vit_acc)*100:.2f}%p")
    
    print("="*80)

# 시각화 실행
create_performance_visualization(training_results, evaluation_results)

# 상세 분석 실행
create_detailed_analysis(evaluation_results)

In [ ]:
# 7. 모델 저장 및 로드 기능

class ModelManager:
    """모델 저장 및 로드 관리 클래스"""
    
    def __init__(self, save_dir="models"):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        
    def save_complete_model(self, model, model_name, scaler=None, processor=None, metadata=None):
        """완전한 모델 정보를 저장"""
        
        model_info = {
            'model_state_dict': model.state_dict(),
            'model_class': model.__class__.__name__,
            'model_config': self._get_model_config(model),
            'scaler': scaler,
            'processor_name': "google/vit-base-patch16-224" if processor else None,
            'metadata': metadata or {}
        }
        
        save_path = os.path.join(self.save_dir, f"{model_name}_complete.pkl")
        torch.save(model_info, save_path)
        
        print(f"💾 {model_name} 완전 모델 저장 완료: {save_path}")
        
    def _get_model_config(self, model):
        """모델 설정 정보 추출"""
        config = {
            'class_name': model.__class__.__name__
        }
        
        if hasattr(model, 'vit') and hasattr(model.vit, 'config'):
            config['vit_model_name'] = "google/vit-base-patch16-224"
        
        return config
        
    def load_model(self, model_name, device='cpu'):
        """저장된 모델 로드"""
        
        save_path = os.path.join(self.save_dir, f"{model_name}_complete.pkl")
        
        if not os.path.exists(save_path):
            print(f"❌ 모델 파일을 찾을 수 없습니다: {save_path}")
            return None
            
        model_info = torch.load(save_path, map_location=device)
        
        print(f"📂 {model_name} 모델 로드 중...")
        print(f"   클래스: {model_info['model_class']}")
        
        return model_info
    
    def save_experiment_results(self, training_results, evaluation_results, dataset_info):
        """실험 결과 종합 저장"""
        
        experiment_data = {
            'training_results': training_results,
            'evaluation_results': {
                k: {
                    'model_name': v['model_name'],
                    'accuracy': v['accuracy'],
                    'precision': v['precision'],
                    'recall': v['recall'],
                    'f1_score': v['f1_score'],
                    'roc_auc': v['roc_auc']
                } for k, v in evaluation_results.items()
            },
            'dataset_info': dataset_info,
            'experiment_date': pd.Timestamp.now().isoformat()
        }
        
        save_path = os.path.join(self.save_dir, "experiment_results.json")
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(experiment_data, f, indent=2, ensure_ascii=False)
            
        print(f"💾 실험 결과 저장 완료: {save_path}")

# 모델 매니저 초기화
model_manager = ModelManager()

# 모든 모델 저장
print("💾 모든 훈련된 모델 저장 중...")

models_to_save = {
    'text_only': text_model,
    'cnn_image': cnn_model,
    'vit_only': vit_model,
    'early_fusion': early_fusion_model,
    'late_fusion': late_fusion_model,
    'attention_fusion': attention_fusion_model
}

for model_name, model in models_to_save.items():
    metadata = {
        'best_val_acc': training_results[model_name]['best_val_acc'],
        'test_accuracy': evaluation_results[model_name]['accuracy'],
        'test_f1_score': evaluation_results[model_name]['f1_score'],
        'model_type': 'unimodal' if model_name in ['text_only', 'cnn_image', 'vit_only'] else 'multimodal'
    }
    
    model_manager.save_complete_model(
        model=model,
        model_name=model_name,
        scaler=data['scaler'],
        processor=ViTImageProcessor.from_pretrained("google/vit-base-patch16-224") if 'vit' in model_name or model_name in ['early_fusion', 'late_fusion', 'attention_fusion'] else None,
        metadata=metadata
    )

# 실험 결과 종합 저장
dataset_info = {
    'num_samples': len(dataset['gender_labels']),
    'num_features': len(dataset['feature_names']),
    'feature_names': dataset['feature_names'],
    'class_names': dataset['class_names'],
    'train_samples': len(data['y_train']),
    'val_samples': len(data['y_val']),
    'test_samples': len(data['y_test'])
}

model_manager.save_experiment_results(training_results, evaluation_results, dataset_info)

print("✅ 모든 모델 및 결과 저장 완료!")

In [ ]:
# 8. 모델 로드 및 추론 예시

def create_inference_demo():
    """저장된 모델을 로드하여 추론하는 데모"""
    
    print("🔍 저장된 모델 로드 및 추론 데모")
    print("="*50)
    
    # 최고 성능 모델 선택
    best_model_name = max(evaluation_results.keys(), key=lambda x: evaluation_results[x]['accuracy'])
    print(f"최고 성능 모델: {best_model_name}")
    
    # 모델 정보 로드
    model_info = model_manager.load_model(best_model_name, device)
    
    if model_info is None:
        print("❌ 모델 로드 실패")
        return
    
    print("✅ 모델 로드 성공!")
    print(f"   클래스: {model_info['model_class']}")
    print(f"   메타데이터: {model_info['metadata']}")
    
    # 새로운 모델 인스턴스 생성 및 가중치 로드
    if best_model_name == 'text_only':
        loaded_model = TextOnlyModel(input_dim=data['X_clinical_train'].shape[1]).to(device)
    elif best_model_name == 'cnn_image':
        loaded_model = CNNImageModel().to(device)
    elif best_model_name == 'vit_only':
        loaded_model = ViTOnlyModel().to(device)
    elif best_model_name == 'early_fusion':
        loaded_model = EarlyFusionModel(clinical_input_dim=data['X_clinical_train'].shape[1]).to(device)
    elif best_model_name == 'late_fusion':
        loaded_model = LateFusionModel(clinical_input_dim=data['X_clinical_train'].shape[1]).to(device)
    elif best_model_name == 'attention_fusion':
        loaded_model = AttentionFusionModel(clinical_input_dim=data['X_clinical_train'].shape[1]).to(device)
    
    loaded_model.load_state_dict(model_info['model_state_dict'])
    loaded_model.eval()
    
    # 테스트 샘플로 추론 시연
    print("\\n🧪 추론 시연:")
    
    # 무작위 테스트 샘플 선택
    sample_idx = np.random.randint(0, len(data['X_clinical_test']))
    
    sample_clinical = torch.FloatTensor(data['X_clinical_test'][sample_idx:sample_idx+1]).to(device)
    sample_image = data['X_images_test'][sample_idx]
    true_label = data['y_test'][sample_idx]
    
    # 이미지 전처리
    if 'vit' in best_model_name or best_model_name in ['early_fusion', 'late_fusion', 'attention_fusion']:
        processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
        sample_image_pil = Image.fromarray(sample_image)
        sample_image_processed = processor(sample_image_pil, return_tensors="pt")['pixel_values'].to(device)
    else:
        sample_image_processed = torch.FloatTensor(sample_image).permute(2, 0, 1).unsqueeze(0).to(device) / 255.0
    
    # 추론 실행
    with torch.no_grad():
        if best_model_name == 'text_only':
            outputs = loaded_model(sample_clinical)
        elif best_model_name == 'cnn_image' or best_model_name == 'vit_only':
            outputs = loaded_model(sample_image_processed)
        else:  # 멀티모달
            outputs = loaded_model(sample_image_processed, sample_clinical)
        
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class = torch.argmax(outputs, dim=1).item()
        confidence = probabilities[0][predicted_class].item()
    
    # 결과 출력
    class_names = data['class_names']
    print(f"   실제 성별: {class_names[true_label]}")
    print(f"   예측 성별: {class_names[predicted_class]}")
    print(f"   예측 신뢰도: {confidence:.4f}")
    print(f"   예측 정확도: {'✅ 정확' if predicted_class == true_label else '❌ 틀림'}")
    
    # 임상 데이터 정보 출력
    print("\\n📊 임상 데이터:")
    feature_names = data['feature_names']
    original_values = model_info['scaler'].inverse_transform(sample_clinical.cpu().numpy())[0]
    
    for feature_name, value in zip(feature_names, original_values):
        print(f"   {feature_name}: {value:.2f}")
    
    # 이미지 시각화
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(sample_image)
    plt.title(f'초음파 이미지\\n실제: {class_names[true_label]}', fontweight='bold')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    prob_values = probabilities[0].cpu().numpy()
    colors = ['pink', 'lightblue']
    bars = plt.bar(class_names, prob_values, color=colors, alpha=0.8)
    plt.title(f'예측 확률\\n예측: {class_names[predicted_class]} ({confidence:.3f})', fontweight='bold')
    plt.ylabel('확률')
    plt.ylim(0, 1)
    
    # 확률 값 표시
    for bar, prob in zip(bars, prob_values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{prob:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.subplot(1, 3, 3)
    # 임상 특징 레이더 차트 (주요 특징만)
    angles = np.linspace(0, 2*np.pi, len(feature_names), endpoint=False).tolist()
    angles += angles[:1]  # 원형으로 만들기
    
    # 정규화된 값 사용
    normalized_values = sample_clinical.cpu().numpy()[0].tolist()
    normalized_values += normalized_values[:1]
    
    plt.polar(angles, normalized_values, 'o-', linewidth=2, alpha=0.8)
    plt.fill(angles, normalized_values, alpha=0.25)
    plt.xticks(angles[:-1], [name.replace('_', '\\n') for name in feature_names], fontsize=8)
    plt.title('임상 특징 프로필', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('models/inference_demo.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return loaded_model

# 추론 데모 실행
demo_model = create_inference_demo()

## 9. 결론 및 향후 연구 방향

### 🎯 실험 결과 요약

본 실험에서는 태아 성별 예측을 위한 다양한 모델들을 비교 분석했습니다:

#### 📊 주요 발견사항:

1. **멀티모달 모델의 우수성**: 단일 모달 모델 대비 멀티모달 모델이 일반적으로 더 높은 성능을 보였습니다.

2. **융합 전략별 성능 차이**: 
   - Attention Fusion: 가장 정교한 특징 융합
   - Early Fusion: 특징 단계에서의 효과적인 결합
   - Late Fusion: 예측 단계에서의 안정적인 결합

3. **모달리티별 기여도**:
   - ViT 기반 이미지 모델: 시각적 특징 학습 우수
   - 텍스트 모델: 임상 수치 데이터의 패턴 학습
   - 융합 모델: 두 모달리티의 상호 보완적 효과

### 🔬 논문계획서 대비 달성 사항:

- ✅ 단일 모달 기반 모델 (텍스트, CNN, ViT) 구현 및 평가
- ✅ 멀티모달 융합 전략 (Early, Late, Attention) 비교
- ✅ 모델별 성능 평가 및 통계적 분석
- ✅ 모델 저장/로드 기능을 통한 재사용 가능성 확보
- ✅ 시각화를 통한 결과 분석 및 해석

### 🚀 향후 연구 방향:

1. **실제 의료 데이터 적용**: AI-Hub 태아 초음파 데이터셋 활용
2. **모델 성능 개선**: 
   - 하이퍼파라미터 최적화
   - 데이터 증강 기법 적용
   - 앙상블 기법 도입

3. **설명 가능한 AI 적용**:
   - Grad-CAM을 통한 주요 영역 시각화
   - SHAP 값을 통한 임상 특징 중요도 분석

4. **임상적 유용성 검증**:
   - 의료진과의 협업을 통한 모델 검증
   - 실제 임상 환경에서의 성능 평가

### 💡 기술적 개선 사항:

- 더 큰 규모의 데이터셋 활용
- 트랜스포머 기반 시계열 모델링 (동영상 데이터)
- 객체 탐지 모델(YOLO, R-CNN)을 통한 관심 영역 추출
- 3D CNN을 활용한 시공간 특징 학습

### 📈 예상 성능 향상 방안:

1. **데이터 품질 개선**: 고해상도 이미지, 정제된 임상 데이터
2. **모델 아키텍처 최적화**: 더 깊은 네트워크, 효율적인 어텐션 메커니즘
3. **전이학습 활용**: 대규모 의료 이미지 데이터로 사전 훈련된 모델 활용
4. **도메인 특화 특징**: 초음파 영상 특성을 고려한 전처리 및 특징 추출

이 실험은 멀티모달 의료 AI 시스템의 가능성을 보여주며, 실제 임상 환경에서의 적용을 위한 기반을 마련했습니다.